In [1]:
# inspiered by https://www.kaggle.com/tunguz/jan-21-tps-h2o-automl

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
from typing import Tuple



C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# main flow
start_time = dt.datetime.now()
print("Started at ", start_time)

Started at  2021-01-30 22:58:22.425989


In [3]:
# Starting H2O

import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.32.0.3
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.1+13-LTS, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\USER-PC\AppData\Local\Temp\tmp220oh13j
  JVM stdout: C:\Users\USER-PC\AppData\Local\Temp\tmp220oh13j\h2o_USER_PC_started_from_python.out
  JVM stderr: C:\Users\USER-PC\AppData\Local\Temp\tmp220oh13j\h2o_USER_PC_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Helsinki
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_USER_PC_l1sndt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
in_kaggle = False


def get_data_file_path(is_in_kaggle: bool) -> Tuple[str, str, str]:
    train_path = ''
    test_path = ''
    sample_submission_path = ''

    if is_in_kaggle:
        # running in Kaggle, inside the competition
        train_path = '../input/tabular-playground-series-jan-2021/train.csv'
        test_path = '../input/tabular-playground-series-jan-2021/test.csv'
        sample_submission_path = '../input/tabular-playground-series-jan-2021/sample_submission.csv'
    else:
        # running locally
        train_path = 'data/train.csv'
        test_path = 'data/test.csv'
        sample_submission_path = 'data/sample_submission.csv'

    return train_path, test_path, sample_submission_path

In [5]:
%%time
# get the training set and labels
train_set_path, test_set_path, sample_subm_path = get_data_file_path(in_kaggle)

df_train = pd.read_csv(train_set_path)
df_test = pd.read_csv(test_set_path)

subm = pd.read_csv(sample_subm_path)

# list of basic raw features
feature_list = [col for col in df_train.columns if col.startswith('cont')]

Wall time: 1.24 s


In [6]:
%%time
# add additional features pre-selected in the previous featurewiz feature importance experiments
def add_extra_features(
    train: pd.DataFrame, 
    test: pd.DataFrame) -> [pd.DataFrame, pd.DataFrame]:
    
    small_val = 0.00001
    for df in [train, test]:
        df['cont2_squared'] = df['cont2'] ** 2
        df['cont3_squared'] = df['cont3'] ** 2
        df['cont9_squared'] = df['cont9'] ** 2
        df['cont12_squared'] = df['cont12'] ** 2
        df['cont14_squared'] = df['cont14'] ** 2
        df['cont7_squared'] = df['cont7'] ** 2
        df['cont13_squared'] = df['cont13'] ** 2
        df['cont8_squared'] = df['cont8'] ** 2
        df['cont11_plus_cont3'] = df['cont11'] + df['cont3']
        df['cont13_plus_cont3'] = df['cont13'] + df['cont3']
        df['cont1_plus_cont4'] = df['cont1'] + df['cont4']
        df['cont5_plus_cont11'] = df['cont5'] + df['cont11']
        df['cont3_plus_cont8'] = df['cont3'] + df['cont8']
        df['cont3_plus_cont14'] = df['cont3'] + df['cont14']
        df['cont4_plus_cont3'] = df['cont4'] + df['cont3']
        df['cont4_plus_cont2'] = df['cont4'] + df['cont2']
        df['cont14_plus_cont11'] = df['cont14'] + df['cont11']
        df['cont14_plus_cont4'] = df['cont14'] + df['cont4']
        df['cont10_minus_cont12'] = df['cont10'] - df['cont12']
        df['cont13_minus_cont2'] = df['cont13'] - df['cont2']
        df['cont11_minus_cont10'] = df['cont11'] - df['cont10']
        df['cont4_minus_cont11'] = df['cont4'] - df['cont11']
        df['cont1_minus_cont6'] = df['cont1'] - df['cont6']
        df['cont5_minus_cont4'] = df['cont5'] - df['cont4']
        df['cont13_minus_cont10'] = df['cont13'] - df['cont10']
        df['cont13_minus_cont6'] = df['cont13'] - df['cont6']
        df['cont10_minus_cont6'] = df['cont10'] - df['cont6']
        df['cont13_minus_cont8'] = df['cont13'] - df['cont8']
        df['cont14_minus_cont13'] = df['cont14'] - df['cont13']
        df['cont9_minus_cont13'] = df['cont9'] - df['cont13']
        df['cont7_minus_cont13'] = df['cont7'] - df['cont13']
        df['cont12_minus_cont8'] = df['cont12'] - df['cont8']
        df['cont2_minus_cont3'] = df['cont2'] - df['cont3']
        df['cont8_minus_cont4'] = df['cont8'] - df['cont4']
        df['cont12_minus_cont6'] = df['cont12'] - df['cont6']
        df['cont5_minus_cont2'] = df['cont5'] - df['cont12']
        df['cont14_minus_cont8'] = df['cont14'] - df['cont8']
        df['cont10_minus_cont9'] = df['cont10'] - df['cont9']
        df['cont1_minus_cont9'] = df['cont1'] - df['cont9']
        df['cont9_minus_cont12'] = df['cont9'] - df['cont12']
        df['cont6_minus_cont11'] = df['cont6'] - df['cont11']
        df['cont6_minus_cont9'] = df['cont6'] - df['cont9']
        df['cont14_minus_cont4'] = df['cont14'] - df['cont4']
        df['cont7_minus_cont11'] = df['cont7'] - df['cont11']
        df['cont1_minus_cont10'] = df['cont1'] - df['cont10']
        df['cont12_minus_cont11'] = df['cont12'] - df['cont11']
        df['cont4_prod_cont3'] = df['cont4'] * df['cont3']
        df['cont4_prod_cont2'] = df['cont4'] * df['cont2']
        df['cont11_prod_cont4'] = df['cont11'] * df['cont4']
        df['cont11_prod_cont3'] = df['cont11'] * df['cont3']
        df['cont13_prod_cont3'] = df['cont13'] * df['cont3']
        df['cont12_prod_cont5'] = df['cont12'] * df['cont5']
        df['cont14_prod_cont11'] = df['cont14'] * df['cont11']
        df['cont8_prod_cont3'] = df['cont8'] * df['cont3']
        df['cont14_prod_cont3'] = df['cont14'] * df['cont3']
        df['cont12_qq_cont10'] = (df['cont12'] + df['cont10']) * (df['cont12'] - df['cont10'])
        df['cont10_qq_cont6'] = (df['cont10'] + df['cont6']) * (df['cont10'] - df['cont6'])
        df['cont1_qq_cont6'] = (df['cont1'] + df['cont6']) * (df['cont1'] - df['cont6'])
        df['cont13_qq_cont10'] = (df['cont13'] + df['cont10']) * (df['cont13'] - df['cont10'])
        df['cont11_qq_cont13'] = (df['cont11'] + df['cont13']) * (df['cont11'] - df['cont13'])
        df['cont4_qq_cont11'] = (df['cont4'] + df['cont11']) * (df['cont4'] - df['cont11'])
        df['cont8_qq_cont10'] = (df['cont8'] + df['cont10']) * (df['cont8'] - df['cont10'])
        df['cont9_qq_cont13'] = (df['cont9'] + df['cont13']) * (df['cont9'] - df['cont13'])
        df['cont13_qq_cont2'] = (df['cont13'] + df['cont2']) * (df['cont13'] - df['cont2'])
        df['cont5_qq_cont9'] = (df['cont5'] + df['cont9']) * (df['cont5'] - df['cont9'])
        df['cont7_qq_cont2'] = (df['cont7'] + df['cont2']) * (df['cont7'] - df['cont2'])
        df['cont1_qq_cont12'] = (df['cont1'] + df['cont12']) * (df['cont1'] - df['cont12'])
        df['cont3_qq_cont11'] = (df['cont3'] + df['cont11']) * (df['cont3'] - df['cont11'])
        df['cont5_qq_cont4'] = (df['cont5'] + df['cont4']) * (df['cont5'] - df['cont4'])
        df['cont10_qq_cont9'] = (df['cont10'] + df['cont9']) * (df['cont10'] - df['cont9'])
        df['cont7_qq_cont9'] = (df['cont7'] + df['cont9']) * (df['cont7'] - df['cont9'])
        df['cont3_qq_cont2'] = (df['cont3'] + df['cont2']) * (df['cont3'] - df['cont2'])
        df['cont1_qq_cont9'] = (df['cont1'] + df['cont9']) * (df['cont1'] - df['cont9'])
        df['cont14_qq_cont8'] = (df['cont14'] + df['cont8']) * (df['cont14'] - df['cont8'])
        df['cont8_qq_cont2'] = (df['cont8'] + df['cont2']) * (df['cont8'] - df['cont2'])
        df['cont5_qq_cont14'] = (df['cont5'] + df['cont14']) * (df['cont5'] - df['cont14'])
        df['cont6_qq_cont12'] = (df['cont6'] + df['cont12']) * (df['cont6'] - df['cont12'])
        df['cont11_qq_cont14'] = (df['cont11'] + df['cont14']) * (df['cont11'] - df['cont14'])
        df['cont12_qq_cont11'] = (df['cont12'] + df['cont11']) * (df['cont12'] - df['cont11'])
        df['cont1_qq_cont10'] = (df['cont1'] + df['cont10']) * (df['cont1'] - df['cont10'])
        df['cont4_qq_cont8'] = (df['cont4'] + df['cont8']) * (df['cont4'] - df['cont8'])
        df['cont10_div2_cont12'] = (df['cont10'] - df['cont12']) / (df['cont10'] + df['cont12'] + small_val)
        df['cont7_div2_cont1'] = (df['cont7'] - df['cont1']) / (df['cont7'] + df['cont1'] + small_val)
        df['cont10_div2_cont11'] = (df['cont10'] - df['cont11']) / (df['cont10'] + df['cont11'] + small_val)
        df['cont10_div2_cont6'] = (df['cont10'] - df['cont6']) / (df['cont10'] + df['cont6'] + small_val)
        df['cont12_div2_cont13'] = (df['cont12'] - df['cont13']) / (df['cont12'] + df['cont13'] + small_val)
        df['cont13_div2_cont8'] = (df['cont13'] - df['cont8']) / (df['cont13'] + df['cont8'] + small_val)
        df['cont9_div2_cont2'] = (df['cont9'] - df['cont2']) / (df['cont9'] + df['cont2'] + small_val)
        df['cont7_div2_cont4'] = (df['cont7'] - df['cont4']) / (df['cont7'] + df['cont4'] + small_val)
        df['cont9_div2_cont3'] = (df['cont9'] - df['cont3']) / (df['cont9'] + df['cont3'] + small_val)
        df['cont13_div2_cont1'] = (df['cont13'] - df['cont1']) / (df['cont13'] + df['cont1'] + small_val)
        df['cont10_div2_cont13'] = (df['cont10'] - df['cont13']) / (df['cont10'] + df['cont13'] + small_val)
        df['cont11_div2_cont3'] = (df['cont11'] - df['cont3']) / (df['cont11'] + df['cont3'] + small_val)
        df['cont7_div2_cont10'] = (df['cont7'] - df['cont10']) / (df['cont7'] + df['cont10'] + small_val)
        df['cont10_div2_cont1'] = (df['cont10'] - df['cont1']) / (df['cont10'] + df['cont1'] + small_val)
        df['cont8_div2_cont11'] = (df['cont8'] - df['cont11']) / (df['cont8'] + df['cont11'] + small_val)
        df['cont11_div2_cont9'] = (df['cont11'] - df['cont9']) / (df['cont11'] + df['cont9'] + small_val)
        df['cont9_div2_cont6'] = (df['cont9'] - df['cont6']) / (df['cont9'] + df['cont6'] + small_val)
        df['cont4_div2_cont14'] = (df['cont4'] - df['cont14']) / (df['cont4'] + df['cont14'] + small_val)
        df['cont4_div2_cont5'] = (df['cont4'] - df['cont5']) / (df['cont4'] + df['cont5'] + small_val)
        df['cont9_div2_cont1'] = (df['cont9'] - df['cont1']) / (df['cont9'] + df['cont1'] + small_val)
        df['cont8_div2_cont4'] = (df['cont8'] - df['cont4']) / (df['cont8'] + df['cont4'] + small_val)
        df['cont14_div2_cont5'] = (df['cont14'] - df['cont5']) / (df['cont14'] + df['cont5'] + small_val)
        df['cont9_div2_cont7'] = (df['cont9'] - df['cont7']) / (df['cont9'] + df['cont7'] + small_val)
        df['cont8_div2_cont14'] = (df['cont8'] - df['cont14']) / (df['cont8'] + df['cont14'] + small_val)
        df['cont6_div2_cont12'] = (df['cont6'] - df['cont12']) / (df['cont6'] + df['cont12'] + small_val)
        df['cont9_div2_cont8'] = (df['cont9'] - df['cont8']) / (df['cont9'] + df['cont8'] + small_val)
        df['cont6_div2_cont7'] = (df['cont6'] - df['cont7']) / (df['cont6'] + df['cont7'] + small_val)
        df['cont10_div2_cont9'] = (df['cont10'] - df['cont9']) / (df['cont10'] + df['cont9'] + small_val)
        df['cont7_div2_cont12'] = (df['cont7'] - df['cont12']) / (df['cont7'] + df['cont12'] + small_val)
        df['cont11_div2_cont12'] = (df['cont11'] - df['cont12']) / (df['cont11'] + df['cont12'] + small_val)
        df['cont11_div2_cont7'] = (df['cont11'] - df['cont7']) / (df['cont11'] + df['cont7'] + small_val)
        df['cont12_div2_cont10'] = (df['cont12'] - df['cont10']) / (df['cont12'] + df['cont10'] + small_val)
        df['cont11_div2_cont10'] = (df['cont11'] - df['cont10']) / (df['cont11'] + df['cont10'] + small_val)
        df['cont6_div2_cont10'] = (df['cont6'] - df['cont10']) / (df['cont6'] + df['cont10'] + small_val)
        df['cont8_div2_cont13'] = (df['cont8'] - df['cont13']) / (df['cont8'] + df['cont13'] + small_val)
        df['cont4_div2_cont8'] = (df['cont4'] - df['cont8']) / (df['cont4'] + df['cont8'] + small_val)
        df['cont13_div2_cont10'] = (df['cont13'] - df['cont10']) / (df['cont13'] + df['cont10'] + small_val)
        df['cont5_div2_cont4'] = (df['cont5'] - df['cont4']) / (df['cont5'] + df['cont4'] + small_val)
        df['cont10_div2_cont7'] = (df['cont10'] - df['cont7']) / (df['cont10'] + df['cont7'] + small_val)
        df['cont3_div2_cont9'] = (df['cont3'] - df['cont9']) / (df['cont3'] + df['cont9'] + small_val)
        df['cont9_div2_cont11'] = (df['cont9'] - df['cont11']) / (df['cont9'] + df['cont11'] + small_val)
        df['cont1_div2_cont9'] = (df['cont1'] - df['cont9']) / (df['cont1'] + df['cont9'] + small_val)
        df['cont3_div2_cont11'] = (df['cont3'] - df['cont11']) / (df['cont3'] + df['cont11'] + small_val)
        df['cont3_div2_cont5'] = (df['cont3'] - df['cont5']) / (df['cont3'] + df['cont5'] + small_val)
        df['cont14_div2_cont8'] = (df['cont14'] - df['cont8']) / (df['cont14'] + df['cont8'] + small_val)
        df['cont9_div2_cont10'] = (df['cont9'] - df['cont10']) / (df['cont9'] + df['cont10'] + small_val)
        df['cont9_div2_cont10'] = (df['cont9'] - df['cont10']) / (df['cont9'] + df['cont10'] + small_val)
        df['cont12_div2_cont11'] = (df['cont12'] - df['cont11']) / (df['cont12'] + df['cont11'] + small_val)
    
    return train, test

# add extra features
# df_train, df_test = add_extra_features(df_train, df_test)

# drop sig_id from train and test sets
df_train = df_train.drop(['id'], axis=1, errors='ignore')
df_test = df_test.drop(['id'], axis=1, errors='ignore')

Wall time: 36 ms


In [7]:
%%time
# subset features and target variable

X = df_train[feature_list] 
X_test = df_test[feature_list] 
y = df_train['target']

Wall time: 28 ms


In [8]:
%%time
# do pre-processing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# cast numpy arrays back to pandas dataframes
X = pd.DataFrame(data=X, columns=feature_list)
X_test = pd.DataFrame(data=X_test, columns=feature_list)
X['target'] = y

Wall time: 1.39 s


In [9]:
%%time
train = h2o.H2OFrame(X)
test = h2o.H2OFrame(X_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Wall time: 17.5 s


In [10]:
x = test.columns
y = 'target'

In [11]:
# Run AutoML for 200 base models, up to 4 h in terms of duration (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=200, seed=47, max_runtime_secs=14400)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
22:59:08.838: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [12]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid__1_AutoML_20210130_225908_model_47,0.490195,0.700139,0.490195,0.585338,0.0796191
GBM_5_AutoML_20210130_225908,0.490394,0.700281,0.490394,0.585271,0.0796292
GBM_grid__1_AutoML_20210130_225908_model_53,0.490663,0.700473,0.490663,0.587303,0.0796588
GBM_grid__1_AutoML_20210130_225908_model_6,0.490711,0.700508,0.490711,0.586037,0.0796557
GBM_grid__1_AutoML_20210130_225908_model_52,0.490839,0.700599,0.490839,0.58704,0.079671
GBM_3_AutoML_20210130_225908,0.491189,0.700849,0.491189,0.587391,0.0796991
GBM_grid__1_AutoML_20210130_225908_model_54,0.491208,0.700863,0.491208,0.585981,0.0796959
GBM_4_AutoML_20210130_225908,0.491323,0.700945,0.491323,0.586666,0.0797052
GBM_2_AutoML_20210130_225908,0.491695,0.701209,0.491695,0.588076,0.0797403
GBM_grid__1_AutoML_20210130_225908_model_16,0.492057,0.701468,0.492057,0.588055,0.0797648


In [13]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid__1_AutoML_20210130_225908_model_47


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,70.0,70.0,876012.0,2.0,15.0,14.657143,3.0,1766.0,991.7714




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.4077463159495112
RMSE: 0.6385501671360765
MAE: 0.533918558661143
RMSLE: 0.07272353821979304
Mean Residual Deviance: 0.4077463159495112

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.4901950676440658
RMSE: 0.700139320167112
MAE: 0.5853380621343189
RMSLE: 0.07961911502862244
Mean Residual Deviance: 0.4901950676440658

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.58533806,0.0021083916,0.58554184,0.58413607,0.58301705,0.5853547,0.58864075
1,mean_residual_deviance,0.49019507,0.0031656206,0.49050486,0.48769432,0.48742908,0.49004692,0.49530017
2,mse,0.49019507,0.0031656206,0.49050486,0.48769432,0.48742908,0.49004692,0.49530017
3,r2,0.087807015,0.0014856353,0.089745626,0.08701511,0.0876401,0.0859144,0.08871985
4,residual_deviance,0.49019507,0.0031656206,0.49050486,0.48769432,0.48742908,0.49004692,0.49530017
5,rmse,0.7001364,0.0022579627,0.70036054,0.69835114,0.6981612,0.7000335,0.70377564
6,rmsle,0.079618044,4.6118637E-4,0.079516165,0.07937904,0.07933526,0.07942551,0.08043427



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2021-01-30 23:40:10,26 min 49.042 sec,0.0,0.733070,0.619374,0.537391
1,,2021-01-30 23:40:11,26 min 49.802 sec,5.0,0.710426,0.600381,0.504705
2,,2021-01-30 23:40:12,26 min 50.522 sec,10.0,0.696009,0.587092,0.484428
3,,2021-01-30 23:40:12,26 min 51.204 sec,15.0,0.685341,0.577021,0.469693
4,,2021-01-30 23:40:13,26 min 51.868 sec,20.0,0.676672,0.568840,0.457885
5,,2021-01-30 23:40:13,26 min 52.458 sec,25.0,0.671199,0.563591,0.450508
6,,2021-01-30 23:40:14,26 min 53.080 sec,30.0,0.665637,0.558439,0.443072
7,,2021-01-30 23:40:15,26 min 53.677 sec,35.0,0.660804,0.553951,0.436662
8,,2021-01-30 23:40:15,26 min 54.354 sec,40.0,0.656183,0.549728,0.430576
9,,2021-01-30 23:40:16,26 min 55.067 sec,45.0,0.653006,0.546853,0.426417



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,cont4,19558.316406,1.000000,0.095545
1,cont3,17909.208984,0.915683,0.087489
2,cont13,16731.908203,0.855488,0.081738
3,cont2,16466.173828,0.841901,0.080440
4,cont14,15127.467773,0.773454,0.073900
5,cont1,14965.779297,0.765188,0.073110
6,cont7,14077.039062,0.719747,0.068769
7,cont12,14001.984375,0.715909,0.068402
8,cont6,13594.057617,0.695053,0.066409
9,cont11,13407.513672,0.685515,0.065498


In [14]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [15]:
preds.as_data_frame().values.flatten()

array([7.98533921, 7.73620547, 7.95771675, ..., 8.19920336, 8.0332086 ,
       7.92750251])

In [16]:
subm['target'] = preds.as_data_frame().values.flatten()
subm.to_csv('h2o_automl_submission.csv', index=False)

In [17]:
print('We are done. That is all, folks!')
finish_time = dt.datetime.now()
print("Finished at ", finish_time)
elapsed = finish_time - start_time
print("Elapsed time: ", elapsed)

We are done. That is all, folks!
Finished at  2021-01-31 01:56:36.719318
Elapsed time:  2:58:14.293329


# Notes

## Baseline Without Scaling

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.410455037942908
RMSE: 0.6406676501454619
MAE: 0.5357954786904653
RMSLE: 0.07296302735787598
Mean Residual Deviance: 0.410455037942908

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.4894685817991209
RMSE: 0.6996203125975695
MAE: 0.5851757939065085
RMSLE: 0.07955961419568029
Mean Residual Deviance: 0.4894685817991209

Public leaderboard score: 

## Baseline With Scaling

ModelMetricsRegression: gbm
** Reported on train data. **

- MSE: 0.4077463159495112
- RMSE: 0.6385501671360765
- MAE: 0.533918558661143
- RMSLE: 0.07272353821979304
Mean Residual Deviance: 0.4077463159495112

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 0.4901950676440658
RMSE: 0.700139320167112
MAE: 0.5853380621343189
RMSLE: 0.07961911502862244
Mean Residual Deviance: 0.4901950676440658

Public leaderboard score: 0.70189